# Import Libraries

In [1]:
!pip install keras-cv-attention-models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.3/618.3 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 21.12.2 requires cupy-cuda115, which is not installed.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.79.0 which is incompatible.
tfx-bsl 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
tensorflow-transform 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
apache-beam 2.44.0 requires 

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import matplotlib as mpl

from tqdm.notebook import tqdm
from multiprocessing import cpu_count
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split
from keras_cv_attention_models import convnext


import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

import os
import time
import pickle
import math
import random
import sys
import cv2
import gc
import datetime

print(f'Tensorflow Version: {tf.__version__}')
print(f'Python Version: {sys.version}')

Tensorflow Version: 2.11.0
Python Version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]


In [3]:
now = datetime.datetime.now().strftime("%d-%b-%Y %H-%M-%S")
np.save(now, np.array([now]))

## MP policy

In [4]:
# # float32 or mixed_float16 (mixed precision: compute float16, variable float32)
# # TPU is fast enough and has enough memory to use float32
# policy = tf.keras.mixed_precision.Policy('float32')
# tf.keras.mixed_precision.set_global_policy(policy)

# print(f'Compute dtype: {tf.keras.mixed_precision.global_policy().compute_dtype}')
# print(f'Variable dtype: {tf.keras.mixed_precision.global_policy().variable_dtype}')

## Matplotlib Config¶


In [5]:
# MatplotLib Global Settings
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['axes.titlesize'] = 24

## Config

In [6]:
# gpu_devices = tf.config.experimental.list_physical_devices('GPU')
# for device in gpu_devices:
#     tf.config.experimental.set_memory_growth(device, True)

## Load Data

In [7]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  '/kaggle/input/fake-images-detecttion-dataset/test-task1/test-task1',
  seed=0,
  image_size=(160, 160),
  batch_size=8,
  labels=None,
  shuffle=False)

Found 7000 files belonging to 1 classes.


# Model

In [8]:
# Seed all random number generators
def seed_everything(seed=0):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything()


In [9]:
def normalize(image):
#     # Repeat channels to create 3 channel images required by pretrained ConvNextV2 models
#     image = tf.repeat(image, repeats=3, axis=3)
    # Cast to float 32
    image = tf.cast(image, tf.float32)
    # Normalize with respect to ImageNet mean/std
    image = tf.keras.applications.imagenet_utils.preprocess_input(image, mode='torch')

    return image

In [10]:
STRATEGY = tf.distribute.MirroredStrategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

N_REPLICAS = STRATEGY.num_replicas_in_sync

In [11]:
N_REPLICAS

1

In [12]:
from keras_cv_attention_models import mobilenetv3

In [13]:
def get_model(path):
    # Inputs, note the names are equal to the dictionary keys in the dataset
    image = tf.keras.layers.Input((160, 160, 3), name='image', dtype=tf.uint8)

    # Normalize Input
    image_norm = normalize(image)

    # CNN Prediction in range [0,1]
    x = mobilenetv3.MobileNetV3Large075(
        input_shape=(160, 160, 3),
        pretrained='imagenet',
        num_classes=0,
    )(image_norm)

    # Average Pooling BxHxWxC -> BxC
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    # Dropout to prevent Overfitting
    x = tf.keras.layers.Dropout(0.1)(x)
    # Output value between [0, 1] using Sigmoid function
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    # Define model with inputs and outputs
    model = tf.keras.models.Model(inputs=image, outputs=outputs)

    # Load pretrained Model Weights
    model.load_weights(path)

    # Set model non-trainable
    model.trainable = False

    # Compile model
    model.compile()

    return model

In [14]:
model = get_model('/kaggle/input/train/model.h5')

    

16621424/16621424 [==============================] - 0s 0us/step
>>>> Load pretrained from: /root/.keras/models/mobilenetv3_large_075_imagenet.h5


# Test

In [15]:
y_true=[]
with open('/kaggle/input/fake-images-detecttion-dataset/label-task1.txt') as f:
    labels = f.readlines()
# for line in lines:
#     splitted_line=line.split(' ')
#     y_true.append(int(splitted_line[1].strip('\n')))
    
    

In [16]:
import glob
image_path = glob.glob('/kaggle/input/fake-images-detecttion-dataset/test-task1/test-task1/*jpg')

In [17]:
parh_len=len('/kaggle/input/fake-images-detecttion-dataset/test-task1/test-task1/')


In [18]:
import cv2

In [19]:
y_true = []  # store true labels
images_to_predict=[]

# iterate over the dataset
for img_path in image_path:   
    for label in labels:
        if img_path[parh_len:] in label:
            splitted_label=label.split(' ')
            y_true.append(int(splitted_label[1].strip('\n')))
            break

    img = cv2.imread(img_path)
    img  = cv2.resize(img, (160, 160))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images_to_predict.append(img)


In [20]:
images_to_predict = np.array(images_to_predict)
y_true = np.array(y_true)

In [21]:
images_to_predict.shape

(7000, 160, 160, 3)

In [22]:
y_true.shape

(7000,)

In [23]:
predicted_labels = model.predict(images_to_predict)

219/219 [==============================] - 9s 14ms/step


In [24]:
predicted_labels[predicted_labels<0.5]=0
predicted_labels[predicted_labels>=0.5]=1

In [25]:
import matplotlib.pyplot as plt
import seaborn as sns


In [26]:
from sklearn.metrics import confusion_matrix

In [27]:
# original test set labels (0 if the image is real; 1 if the image is Deepfake), However here we have it inverted

In [28]:
correct_labels = np.array(y_true)


In [29]:
correct_labels = 1- correct_labels

In [30]:
correct_labels.shape

(7000,)

In [31]:
predicted_labels.shape

(7000, 1)

In [32]:
confusion_matrix(correct_labels, predicted_labels.flatten())

array([[4048,  952],
       [ 368, 1632]])

In [33]:
from sklearn.metrics import classification_report
print(classification_report(correct_labels, predicted_labels.flatten()))

              precision    recall  f1-score   support

           0       0.92      0.81      0.86      5000
           1       0.63      0.82      0.71      2000

    accuracy                           0.81      7000
   macro avg       0.77      0.81      0.79      7000
weighted avg       0.84      0.81      0.82      7000

